# ITCS 6162: Data Mining - Programming Assignment

**In this assignment, you will explore data analysis, recommendation algorithms, and graph-based techniques using the MovieLens dataset. Your tasks will range from basic data exploration to advanced recommendation models, including:**
- Data manipulation with pandas
- User-item collaborative filtering
- Similarity-based recommendation models
- A Pixie-inspired Graph-based recommendation using adjacency lists with weighted random walks (without using NetworkX)


#### **Dataset Files:**
- **`u.data`**: User-movie ratings (`user_id  movie_id  rating  timestamp`)
- **`u.item`**: Movie metadata (`movie_id | title | release date | IMDB_website`)
- **`u.user`**: User demographics (`user_id | age | gender | occupation | zip_code`)

## **Part 1: Exploring and Cleaning Data**

### Inspecting the Dataset Format

The dataset is not in a traditional CSV format. To examine its structure, use the following shell command to display the first 10 lines of the file:

```sh
!head <file_name>


**In the cells given below. Write the code to read the files.**

In [16]:
# u.data
with open ("u.data", "r") as file:
  lines = file.readlines()
  for line in range(10):
    print(lines[line].strip())

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


In [41]:
# u.item
with open ("u.item", "r") as file:
  count = 0
  for line in file:
    count += 1
    print(line.strip())
    if count >= 10:
      break

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

In [20]:
# u.user
with open ("u.user", "r") as file:
  lines = file.readlines()
  for line in range(10):
    print(lines[line].strip())

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


#### Loading the Dataset with Pandas

Use **pandas** to load the dataset into a DataFrame for analysis. Follow these steps:  

1. Import the necessary library: `pandas`.  
2. Use `pd.read_csv()` (or an appropriate function) to read the dataset file.  
3. Ensure the dataset is loaded with the correct delimiter (e.g., `','`, `'\t'`,`'|'` , or another separator if needed).  
4. Select and display the first few rows using `.head()`.

Ensure that:  

- The `ratings` dataset is read from `"u.data"` using tab (`'\t'`) as a separator and column names (`"user_id"`, `"movie_id"`, `"rating"` and `"timestamp"`).  
- The `movies` dataset is read from `"u.item"` using `'|'` as a separator, use columns (`0`, `1`, `2`), encoding (`"latin-1"`) and name the columns (`movie_id`, `title`, and `release_date`).  
- The `users` dataset is read from `"u.user"` using `'|'` as a separator, use columns (`0`, `1`, `2`, `3`) and name the columns (`user_id`, `age`, `gender`, and `occupation`).

In [2]:
# ratings
import pandas as pd
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
print(ratings)

       user_id  movie_id  rating  timestamp
0          196       242       3  881250949
1          186       302       3  891717742
2           22       377       1  878887116
3          244        51       2  880606923
4          166       346       1  886397596
...        ...       ...     ...        ...
99995      880       476       3  880175444
99996      716       204       5  879795543
99997      276      1090       1  874795795
99998       13       225       2  882399156
99999       12       203       3  879959583

[100000 rows x 4 columns]


In [3]:
# movies
import pandas as pd
movies = pd.read_csv('u.item', sep='|', names=['movie_id', 'title', 'release_date'], usecols=[0, 1, 2], encoding="latin-1")
print(movies)

      movie_id                                      title release_date
0            1                           Toy Story (1995)  01-Jan-1995
1            2                           GoldenEye (1995)  01-Jan-1995
2            3                          Four Rooms (1995)  01-Jan-1995
3            4                          Get Shorty (1995)  01-Jan-1995
4            5                             Copycat (1995)  01-Jan-1995
...        ...                                        ...          ...
1677      1678                          Mat' i syn (1997)  06-Feb-1998
1678      1679                           B. Monkey (1998)  06-Feb-1998
1679      1680                       Sliding Doors (1998)  01-Jan-1998
1680      1681                        You So Crazy (1994)  01-Jan-1994
1681      1682  Scream of Stone (Schrei aus Stein) (1991)  08-Mar-1996

[1682 rows x 3 columns]


In [4]:
# users
import pandas as pd
users = pd.read_csv('u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation'], usecols=[0, 1, 2, 3])
print(users)

     user_id  age gender     occupation
0          1   24      M     technician
1          2   53      F          other
2          3   23      M         writer
3          4   24      M     technician
4          5   33      F          other
..       ...  ...    ...            ...
938      939   26      F        student
939      940   32      M  administrator
940      941   20      M        student
941      942   48      F      librarian
942      943   22      M        student

[943 rows x 4 columns]


**Note:** As a **Bonus** task save the `ratings`, `movies` and `users` dataframe created into a `.csv` file format. <br>
**Hint:** Use the `to_csv()` function in pandas to save these DataFrames as CSV files.

In [5]:
# ratings
import pandas as pd
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
ratings.to_csv('ratings.csv')

In [6]:
# movies
import pandas as pd
movies = pd.read_csv('u.item', sep='|', names=['movie_id', 'title', 'release_date'], usecols=[0, 1, 2], encoding="latin-1")
movies.to_csv('movies.csv')

In [7]:
# users
import pandas as pd
users = pd.read_csv('u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation'], usecols=[0, 1, 2, 3])
users.to_csv('users.csv')

**Display the first 10 rows of each file.**

In [8]:
# ratings
import pandas as pd
ratings = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
print(ratings.head(10))

   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596
5      298       474       4  884182806
6      115       265       2  881171488
7      253       465       5  891628467
8      305       451       3  886324817
9        6        86       3  883603013


In [9]:
# movies
import pandas as pd
movies = pd.read_csv('u.item', sep='|', names=['movie_id', 'title', 'release_date'], usecols=[0, 1, 2], encoding="latin-1")
print(movies.head(10))

   movie_id                                              title release_date
0         1                                   Toy Story (1995)  01-Jan-1995
1         2                                   GoldenEye (1995)  01-Jan-1995
2         3                                  Four Rooms (1995)  01-Jan-1995
3         4                                  Get Shorty (1995)  01-Jan-1995
4         5                                     Copycat (1995)  01-Jan-1995
5         6  Shanghai Triad (Yao a yao yao dao waipo qiao) ...  01-Jan-1995
6         7                              Twelve Monkeys (1995)  01-Jan-1995
7         8                                        Babe (1995)  01-Jan-1995
8         9                            Dead Man Walking (1995)  01-Jan-1995
9        10                                 Richard III (1995)  22-Jan-1996


In [10]:
# users
import pandas as pd
users = pd.read_csv('u.user', sep='|', names=['user_id', 'age', 'gender', 'occupation'], usecols=[0, 1, 2, 3])
print(users.head(10))

   user_id  age gender     occupation
0        1   24      M     technician
1        2   53      F          other
2        3   23      M         writer
3        4   24      M     technician
4        5   33      F          other
5        6   42      M      executive
6        7   57      M  administrator
7        8   36      M  administrator
8        9   29      M        student
9       10   53      M         lawyer


### Data Cleaning and Exploration with Pandas  

After loading the dataset, it’s important to clean and explore the data to ensure consistency and accuracy. Below are key **pandas** functions for cleaning and understanding the dataset.

#### 1. Handle Missing Values  
- `df.dropna()` – Removes rows with missing values.  
- `df.fillna(value)` – Fills missing values with a specified value.  

#### 2. Remove Duplicates  
- `df.drop_duplicates()` – Drops duplicate rows from the dataset.  

#### 3. Handle Incorrect Data Types  
- `df.astype(dtype)` – Converts columns to the appropriate data type.  

#### 4. Filter Outliers (if applicable)  
- `df[df['column_name'] > threshold]` – Filters rows based on a condition.  

#### 5. Rename Columns (if needed)  
- `df.rename(columns={'old_name': 'new_name'})` – Renames columns for clarity.  

#### 6. Reset Index  
- `df.reset_index(drop=True, inplace=True)` – Resets the index after cleaning.  

### Data Exploration Functions  

To better understand the dataset, use these **pandas** functions:  

- `df.shape` – Returns the number of rows and columns in the dataset.  
- `df.nunique()` – Displays the number of unique values in each column.  
- `df['column_name'].unique()` – Returns unique values in a specific column.  

**Example Usage in Pandas:**  
```python
import pandas as pd

# Load dataset
df = pd.read_csv("your_file.csv")

# Drop missing values
df_cleaned = df.dropna()

# Remove duplicate rows
df_cleaned = df_cleaned.drop_duplicates()

# Convert 'timestamp' column to datetime format
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

# Display dataset shape
print("Dataset shape:", df_cleaned.shape)

# Display number of unique values in each column
print("Unique values per column:\n", df_cleaned.nunique())

# Display unique movie IDs
print("Unique movie IDs:", df_cleaned['movie_id'].unique()[:10])  # Show first 10 unique movie IDs


**Note:** The functions mentioned above are some of the widely used **pandas** functions for data cleaning and exploration. However, it is not necessary that all of these functions will be required in the exercises below. Use them as needed based on the dataset and the specific tasks.

**Convert Timestamps into Readable dates.**

In [11]:
# ratings
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'])
print(ratings)

       user_id  movie_id  rating                     timestamp
0          196       242       3 1970-01-01 00:00:00.881250949
1          186       302       3 1970-01-01 00:00:00.891717742
2           22       377       1 1970-01-01 00:00:00.878887116
3          244        51       2 1970-01-01 00:00:00.880606923
4          166       346       1 1970-01-01 00:00:00.886397596
...        ...       ...     ...                           ...
99995      880       476       3 1970-01-01 00:00:00.880175444
99996      716       204       5 1970-01-01 00:00:00.879795543
99997      276      1090       1 1970-01-01 00:00:00.874795795
99998       13       225       2 1970-01-01 00:00:00.882399156
99999       12       203       3 1970-01-01 00:00:00.879959583

[100000 rows x 4 columns]


**Check for Missing Values**

In [12]:
# ratings
ratings_cleaned = ratings.dropna()
ratings_cleaned = ratings_cleaned.drop_duplicates()
print(ratings_cleaned)

       user_id  movie_id  rating                     timestamp
0          196       242       3 1970-01-01 00:00:00.881250949
1          186       302       3 1970-01-01 00:00:00.891717742
2           22       377       1 1970-01-01 00:00:00.878887116
3          244        51       2 1970-01-01 00:00:00.880606923
4          166       346       1 1970-01-01 00:00:00.886397596
...        ...       ...     ...                           ...
99995      880       476       3 1970-01-01 00:00:00.880175444
99996      716       204       5 1970-01-01 00:00:00.879795543
99997      276      1090       1 1970-01-01 00:00:00.874795795
99998       13       225       2 1970-01-01 00:00:00.882399156
99999       12       203       3 1970-01-01 00:00:00.879959583

[100000 rows x 4 columns]


In [13]:
# movies
movies_cleaned = movies.dropna().drop_duplicates()
print(movies_cleaned)

      movie_id                                      title release_date
0            1                           Toy Story (1995)  01-Jan-1995
1            2                           GoldenEye (1995)  01-Jan-1995
2            3                          Four Rooms (1995)  01-Jan-1995
3            4                          Get Shorty (1995)  01-Jan-1995
4            5                             Copycat (1995)  01-Jan-1995
...        ...                                        ...          ...
1677      1678                          Mat' i syn (1997)  06-Feb-1998
1678      1679                           B. Monkey (1998)  06-Feb-1998
1679      1680                       Sliding Doors (1998)  01-Jan-1998
1680      1681                        You So Crazy (1994)  01-Jan-1994
1681      1682  Scream of Stone (Schrei aus Stein) (1991)  08-Mar-1996

[1681 rows x 3 columns]


In [14]:
# users
users_cleaned = users.dropna()
users_cleaned = users_cleaned.drop_duplicates()
print(users_cleaned)

     user_id  age gender     occupation
0          1   24      M     technician
1          2   53      F          other
2          3   23      M         writer
3          4   24      M     technician
4          5   33      F          other
..       ...  ...    ...            ...
938      939   26      F        student
939      940   32      M  administrator
940      941   20      M        student
941      942   48      F      librarian
942      943   22      M        student

[943 rows x 4 columns]


**Print the total number of users, movies, and ratings.**

In [15]:
print(f"Total Users: { users_cleaned['user_id'].nunique() }")
print(f"Total Movies: { movies_cleaned['movie_id'].nunique() }")
print(f"Total Ratings: { ratings_cleaned.shape[0] }")

Total Users: 943
Total Movies: 1681
Total Ratings: 100000


## **Part 2: Collaborative Filtering-Based Recommendation**

### **Create a User-Item Matrix**

#### Instructions for Creating a User-Movie Rating Matrix

In this exercise, you will create a user-movie rating matrix using **pandas**. This matrix will represent the ratings that users have given to different movies.

1. **Dataset Overview**:  
   The dataset has already been loaded. It includes the following key columns:
   - `user_id`: The ID of the user.
   - `movie_id`: The ID of the movie.
   - `ratings`: The rating the user gave to the movie.

2. **Create the User-Movie Rating Matrix**:  
   Use the **`pivot()`** function in **pandas** to reshape the data. Your goal is to create a matrix where:
   - Each **row** represents a **user**.
   - Each **column** represents a **movie**.
   - Each **cell** contains the **rating** that the user has given to the movie.

   Specify the following parameters for the `pivot()` function:
   - **`index`**: The `user_id` column (this will define the rows).
   - **`columns`**: The `movie_id` column (this will define the columns).
   - **`values`**: The `rating` column (this will fill the matrix with ratings).

3. **Inspect the Matrix**:  
   After creating the matrix, examine the first few rows of the resulting matrix to ensure it has been constructed correctly.

4. **Handle Missing Values**:  
   It's likely that some users have not rated every movie, resulting in `NaN` values in the matrix. You will need to handle these missing values. Consider the following options:
   - **Fill with 0**: If you wish to represent missing ratings as zeros (indicating no rating).
   - **Fill with the average rating**: Alternatively, replace missing values with the average rating for each movie.

**Create the user-movie rating matrix using the `pivot()` function.**

In [16]:
user_movie_matrix = ratings_cleaned.pivot(index='user_id', columns='movie_id', values='rating')

**Display the matrix to verify the transformation.**

In [17]:
print(user_movie_matrix)

movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2          4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5          4.0   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...        ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
939        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   5.0   NaN  ...   
940        NaN   NaN   NaN   2.0   NaN   NaN   4.0   5.0   3.0   NaN  ...   
941        5.0   NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN  ...   
942        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
943        NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN   3.0   NaN  ...   

### **User-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement a **user-based collaborative filtering** movie recommendation system using the **Movie dataset**. The goal is to recommend movies to a user based on the preferences of similar users.

##### **Step 1: Import Required Libraries**
Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing user similarity
```

##### **Step 2: Compute User-User Similarity**
- We will use **cosine similarity** to measure how similar each pair of users is based on their movie ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.

##### **Instructions:**
1. Fill missing values with `0` using `.fillna(0)`.
2. Compute similarity using `cosine_similarity()`.
3. Convert the result into a **Pandas DataFrame**, with users as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)
```

##### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies_for_user(user_id, num=5)` to recommend movies for a given user.

##### **Function Inputs:**
- `user_id`: The target user for whom we need recommendations.
- `num`: The number of movies to recommend (default is 5).

##### **Function Steps:**
1. Find **similar users**:
   - Retrieve the similarity scores for the given `user_id`.
   - Sort them in **descending** order (highest similarity first).
   - Exclude the user themselves.
   
2. Get the **movie ratings** from these similar users.

3. Compute the **average rating** for each movie based on these users' preferences.

4. Sort the movies in **descending order** based on the computed average ratings.

5. Retrieve the **top `num` recommended movies**.

6. Map **movie IDs** to their **titles** using the `movies` DataFrame.

7. Return the results as a **Pandas DataFrame** with rankings.

##### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'Ranking': range(1, num+1),
    'Movie Name': movie_names     
})
result_df.set_index('Ranking', inplace=True)
```

#### **Example: User-Based Collaborative Filtering**
```python
recommend_movies_for_user(10, num = 5)
```
**Output:**
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | In the Company of Men (1997)   |
| 2       | Misérables, Les (1995)         |
| 3       | Thin Blue Line, The (1988)     |
| 4       | Braindead (1992)               |
| 5       | Boys, Les (1997)               |


In [44]:
# Code the function here
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing user similarity

user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_sim_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

def recommend_movies_for_user(user_id, num=5):
    similar_users = user_sim_df.loc[user_id].drop(user_id).sort_values(ascending=False)

    # Step 2: Get movie ratings from similar users
    top_similar_users = similar_users.index
    similar_users_ratings = ratings_cleaned[ratings_cleaned['user_id'].isin(top_similar_users)]

    # Step 3: Average ratings by movie from similar users
    avg_ratings = similar_users_ratings.groupby('movie_id')['rating'].mean()

    # Step 4: Sort by average rating descending
    sorted_recommendations = avg_ratings.sort_values(ascending=False).head(num)

    # Step 6: Map movieId to title
    recommendations = pd.DataFrame({
        'Movie Name': sorted_recommendations.index,
        'Ratings': sorted_recommendations.values
    })
    recommendations = pd.merge(recommendations, movies_cleaned, left_on='Movie Name', right_on='movie_id')

    # Step 7: Return with ranking
    recommendations['Ranking'] = recommendations['Ratings'].rank(method='first', ascending=False).astype(int)
    recommendations = recommendations.sort_values('Ranking')

    return recommendations[['Ranking', 'title']]

recommendations = recommend_movies_for_user(10, 5)
print(recommendations)


   Ranking                                              title
0        1                                    Star Kid (1997)
1        2                          Santa with Muscles (1996)
2        3  Entertaining Angels: The Dorothy Day Story (1996)
3        4                                 Prefontaine (1997)
4        5         Marlene Dietrich: Shadow and Light (1996) 


### **Item-Based Collaborative Filtering Recommender System**

#### **Objective**
In this task, you will implement an **item-based collaborative filtering** recommendation system using the **Movie dataset**. The goal is to recommend movies similar to a given movie based on user rating patterns.

#### **Step 1: Import Required Libraries**
Although we have done this part already in the previous task but just to emphasize the importance reiterrating this part.

Before starting, ensure you have the necessary libraries installed. Use the following imports:

```python
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing item similarity
```

#### **Step 2: Compute Item-Item Similarity**
- We will use **cosine similarity** to measure how similar each pair of movies is based on their user ratings.
- Since `cosine_similarity` does not handle missing values (NaN), replace them with `0` before computation.
- Unlike user-based filtering, we need to **transpose** (`.T`) the `user_movie_matrix` because we want similarity between movies (columns) instead of users (rows).

##### **Instructions:**
1. Transpose the user-movie matrix using `.T` to make movies the rows.
2. Fill missing values with `0` using `.fillna(0)`.
3. Compute similarity using `cosine_similarity()`.
4. Convert the result into a **Pandas DataFrame**, with movies as both row and column labels.

##### **Hint:**  
You can achieve this using the following approach:

```python
item_similarity = cosine_similarity(user_movie_matrix.T.fillna(0))
item_sim_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)
```

#### **Step 3: Implement the Recommendation Function**
Now, implement the function `recommend_movies(movie_name, num=5)` to recommend movies similar to a given movie.

##### **Function Inputs:**
- `movie_name`: The target movie for which we need recommendations.
- `num`: The number of similar movies to recommend (default is 5).

##### **Function Steps:**
1. Find the **movie_id** corresponding to the given `movie_name` in the `movies` DataFrame.
2. If the movie is not found, return an appropriate message.
3. Extract the **similarity scores** for this movie from `item_sim_df`.
4. Sort the movies in **descending order** based on similarity (excluding the movie itself).
5. Retrieve the **top `num` similar movies**.
6. Map **movie IDs** to their **titles** using the `movies` DataFrame.
7. Return the results as a **Pandas DataFrame** with rankings.

#### **Step 4: Return the Final Recommendation List**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

##### **Hint:** Your final DataFrame should be created like this:
```python
result_df = pd.DataFrame({
    'ranking': range(1, num+1),
    'movie_name': movie_names
})
result_df.set_index('ranking', inplace=True)
```

#### **Example: Item-Based Collaborative Filtering**
```python
recommend_movies("Jurassic Park (1993)", num=5)
```
**Output:**
```
| Ranking | Movie Name                               |
|---------|------------------------------------------|
| 1       | Top Gun (1986)                           |
| 2       | Empire Strikes Back, The (1980)          |
| 3       | Raiders of the Lost Ark (1981)           |
| 4       | Indiana Jones and the Last Crusade (1989)|
| 5       | Speed (1994)                             |


In [72]:
# Code the function here
import pandas as pd  # For handling data
import numpy as np   # For numerical computations
from sklearn.metrics.pairwise import cosine_similarity  # For computing user similarity

item_similarity = cosine_similarity(user_movie_matrix.T.fillna(0))
item_sim_df = pd.DataFrame(item_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

def recommend_movies(movie_name, num=5):
    # Step 1 & 2
    movie_id = movies_cleaned[movies_cleaned['title'].str.lower() == movie_name.lower()]
    if movie_id.empty:
        return f"Movie '{movie_name}' not found in the dataset."
    movie_id = movie_id.iloc[0]['movie_id']
    
    # Step 3
    if movie_id not in item_sim_df.index:
        return f"No similarity data available for movie '{movie_name}'."
    similar_items = item_sim_df[movie_id].drop(movie_id)

    # Step 4, 5
    sorted_item_ratings = similar_items.sort_values(ascending=False)
    top_item_ratings = sorted_item_ratings.head(num)

    # Step 6
    movie_recommendations = pd.DataFrame({
        'Ratings': top_item_ratings.values,
        'Movie Name': top_item_ratings.index
    })
    movie_recommendations = pd.merge(movie_recommendations, movies_cleaned, left_on='Movie Name', right_on='movie_id')

    # Step 7
    movie_recommendations['Ranking'] = movie_recommendations['Ratings'].rank(method='first', ascending=False).astype(int)
    movie_recommendations = movie_recommendations.sort_values('Ranking')

    return movie_recommendations[['Ranking', 'title']]

movie_recommendations = recommend_movies("Jurassic Park (1993)", num=5)
print(movie_recommendations)


   Ranking                                      title
0        1                             Top Gun (1986)
1        2                               Speed (1994)
2        3             Raiders of the Lost Ark (1981)
3        4            Empire Strikes Back, The (1980)
4        5  Indiana Jones and the Last Crusade (1989)


## **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm)**

### **Adjacency List**

#### **Objective**
In this task, you will preprocess the Movie dataset and construct a **graph representation** where:
- **Users** are connected to the movies they have rated.
- **Movies** are connected to users who have rated them.
  
This graph structure will help in exploring **user-movie relationships** for recommendations.

#### **Step 1: Merge Ratings with Movie Titles**
Since we have **movie IDs** in the ratings dataset but need human-readable movie titles, we will:
1. Merge the `ratings` DataFrame with the `movies` DataFrame using the `'movie_id'` column.
2. This allows each rating to be associated with a **movie title**.

#### **Hint:**
Use the following Pandas operation to merge:
```python
ratings = ratings.merge(movies, on='movie_id')
```


#### **Step 2: Aggregate Ratings**
Since multiple users may rate the same movie multiple times, we:
1. Group the dataset by `['user_id', 'movie_id', 'title']`.
2. Compute the **mean rating** for each movie by each user.
3. Reset the index to ensure we maintain a clean DataFrame structure.

#### **Hint:**  
Use `groupby()` and `mean()` as follows:
```python
ratings = ratings.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()
```

#### **Step 3: Normalize Ratings**
Since different users have different rating biases, we normalize ratings by:
1. **Computing each user's mean rating**.
2. **Subtracting the mean rating** from each individual rating.

#### **Instructions:**
- Use `groupby('user_id')` to group ratings by users.
- Apply `transform(lambda x: x - x.mean())` to adjust ratings.

#### **Hint:**  
Normalize ratings using:
```python
ratings['rating'] = ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())
```
This ensures each user’s ratings are centered around zero, making similarity calculations fairer.

#### **Step 4: Construct the Graph Representation**
We represent the user-movie interactions as an **undirected graph** using an **adjacency list**:
- Each **user** is a node connected to movies they rated.
- Each **movie** is a node connected to users who rated it.

#### **Graph Construction Steps:**
1. Initialize an empty dictionary `graph = {}`.
2. Iterate through the **ratings dataset**.
3. For each `user_id` and `movie_id` pair:
   - Add the movie to the user’s set of connections.
   - Add the user to the movie’s set of connections.

#### **Hint:**  
The following code builds the graph:

```python
graph = {}
for _, row in ratings.iterrows():
    user, movie = row['user_id'], row['movie_id']
    if user not in graph:
        graph[user] = set()
    if movie not in graph:
        graph[movie] = set()
    graph[user].add(movie)
    graph[movie].add(user)
```

This results in a **bipartite graph**, where:
- **Users** are connected to multiple movies.
- **Movies** are connected to multiple users.

#### **Step 5: Understanding the Graph**
- **Nodes** in the graph represent **users and movies**.
- **Edges** exist between a user and a movie **if the user has rated the movie**.
- This structure allows us to find **users with similar movie tastes** and **movies frequently watched together**.

#### **Exploring the Graph**
- **Find a user’s rated movies:**  
  ```python
  user_id = 1
  print(graph[user_id])  # Movies rated by user 1
  ```

- **Find users who rated a movie:**  
  ```python
  movie_id = 50
  print(graph[movie_id])  # Users who rated movie 50
  ```

In [80]:
# Code the function here
# Step 1
ratings_merged = ratings_cleaned.merge(movies_cleaned, on='movie_id')

# Step 2
new_ratings = ratings_merged.groupby(['user_id', 'movie_id', 'title'])['rating'].mean().reset_index()

# Step 3
new_ratings['rating'] = new_ratings.groupby('user_id')['rating'].transform(lambda x: x - x.mean())

# Step 4
graph = {}
for _, row in new_ratings.iterrows():
    user, movie = row['user_id'], row['movie_id']
    if user not in graph:
        graph[user] = set()
    if movie not in graph:
        graph[movie] = set()
    graph[user].add(movie)
    graph[movie].add(user)

user_id = 4
print(graph[user_id])

movie_id = 36
print(graph[movie_id])

{1, 514, 7, 10, 11, 12, 13, 524, 527, 16, 18, 19, 532, 533, 22, 535, 537, 538, 543, 551, 554, 43, 559, 49, 50, 561, 562, 59, 62, 64, 577, 579, 77, 591, 592, 593, 83, 84, 87, 600, 92, 94, 608, 99, 102, 618, 109, 621, 622, 625, 115, 627, 629, 638, 640, 130, 642, 643, 645, 648, 650, 653, 654, 655, 144, 659, 151, 664, 666, 158, 671, 160, 682, 690, 189, 194, 707, 197, 198, 709, 712, 201, 715, 716, 207, 210, 216, 218, 219, 221, 222, 738, 232, 233, 234, 747, 748, 749, 757, 758, 249, 763, 764, 253, 256, 258, 771, 260, 774, 264, 268, 780, 271, 786, 276, 788, 790, 279, 280, 795, 796, 286, 287, 288, 291, 804, 293, 294, 295, 805, 297, 299, 300, 301, 303, 308, 823, 312, 826, 315, 318, 320, 833, 324, 326, 327, 328, 329, 334, 846, 336, 339, 851, 342, 343, 344, 345, 346, 347, 854, 860, 352, 864, 354, 356, 357, 358, 359, 360, 361, 362, 363, 870, 871, 875, 880, 881, 882, 883, 373, 374, 886, 889, 378, 379, 896, 385, 387, 389, 903, 393, 394, 398, 913, 916, 405, 406, 407, 919, 411, 412, 416, 417, 421, 933,

### **Implement Weighted Random Walks**

#### **Random Walk-Based Movie Recommendation System (Weighted Pixie)**

#### **Objective**
In this task, you will implement a **random-walk-based recommendation algorithm** using the **Weighted Pixie** method. This technique uses a **user-movie bipartite graph** to recommend movies by simulating a random walk from a given user or movie.

#### **Step 1: Import Required Libraries**
Make sure you have the necessary libraries:

```python
import random  # For random walks
import pandas as pd  # For handling data
```

#### **Step 2: Implement the Random Walk Algorithm**
Your task is to **simulate a random walk** from a given starting point in the **bipartite user-movie graph**.

##### **Hints for Implementation**
- Start from **either a user or a movie**.
- At each step, **randomly move** to a connected node.
- Keep track of **how many times each movie is visited**.
- After completing the walk, **rank movies by visit count**.

#### **Step 3: Implement User-Based Recommendation**
**Hints:**
- Check if the `user_id` exists in the `graph`.
- Start a loop that runs for `walk_length` steps.
- Randomly pick a **connected node** (user or movie).
- Track how many times each **movie** is visited.
- Sort movies by visit frequency and return the **top N**.

#### **Step 4: Implement Movie-Based Recommendation**
**Hints:**
- Find the `movie_id` corresponding to the given `movie_name`.
- Ensure the movie exists in the `graph`.
- Start a random walk from that movie.
- Follow the same **tracking and ranking** process as the user-based version.

**Note:**  
**Your task:** Implement a function `weighted_pixie_recommend(user_id, walk_length=15, num=5)` or `weighted_pixie_recommend(movie_name, walk_length=15, num=5)`.  
**Implement either Step 3 or Step 4.**

#### **Step 5: Running Your Recommendation System**
Once your function is implemented, test it by calling:

##### **Example: User-Based Recommendation**
```python
weighted_pixie_recommend(1, walk_length=15, num=5)
```
| Ranking | Movie Name                     |
|---------|--------------------------------|
| 1       | My Own Private Idaho (1991)   |
| 2       | Aladdin (1992)                |
| 3       | 12 Angry Men (1957)           |
| 4       | Happy Gilmore (1996)          |
| 5       | Copycat (1995)                |


##### **Example: Movie-Based Recommendation**
```python
weighted_pixie_recommend("Jurassic Park (1993)", walk_length=10, num=5)
```
| Ranking | Movie Name                           |
|---------|-------------------------------------|
| 1       | Rear Window (1954)                 |
| 2       | Great Dictator, The (1940)         |
| 3       | Field of Dreams (1989)             |
| 4       | Casablanca (1942)                  |
| 5       | Nightmare Before Christmas, The (1993) |


#### **Step 6: Understanding the Results**
Your function should return a **DataFrame** structured as follows:

| Ranking | Movie Name |
|---------|-----------|
| 1       | Movie A   |
| 2       | Movie B   |
| 3       | Movie C   |
| 4       | Movie D   |
| 5       | Movie E   |

Each movie is ranked based on **how frequently it was visited** during the walk.

#### **Experiment with Different Parameters**
- Try different **`walk_length`** values and observe how it changes recommendations.
- Adjust the number of recommended movies (`num`).

In [96]:
# Code the function here
import random  # For random walks
import pandas as pd  # For handling data

def weighted_pixie_recommend(user_id, walk_length=15, num=5):
    if user_id not in ratings_cleaned['user_id'].values:
        return f"User '{user_id}' not found."
    
    # User based recommendation
    user_to_movies = ratings_cleaned.groupby('user_id')['movie_id'].apply(list).to_dict()
    movie_to_users = ratings_cleaned.groupby('movie_id')['user_id'].apply(list).to_dict()
    
    # Random walk algorithm
    current_node = user_id
    movies_visited = {}
    for _ in range(walk_length):
        if current_node in user_to_movies:
            neighbors = user_to_movies.get(current_node, [])
        else:
            neighbors = movie_to_users.get(current_node, [])
        if not neighbors:
            break
        next_node = random.choice(neighbors)
        if next_node in movie_to_users:
            movies_visited[next_node] = movies_visited.get(next_node, 0) + 1
        current_node = next_node

    # Sort movies by visit frequency
    sorted_movies = sorted(movies_visited.items(), key=lambda x: x[1], reverse=True)[:num]
    recommend = pd.DataFrame(sorted_movies, columns=['movie_id', 'visit_count'])
    recommend = recommend.merge(movies_cleaned, on='movie_id')
    recommend['Ranking'] = recommend['visit_count'].rank(method='first', ascending=False).astype(int)
    recommend = recommend.sort_values('Ranking')

    return recommend[['Ranking', 'title', 'visit_count']]

recommend = weighted_pixie_recommend(3, 15, 5)
print(recommend)

   Ranking                                title  visit_count
0        1              Immortal Beloved (1994)            2
1        2                  Dante's Peak (1997)            1
2        3                     Liar Liar (1997)            1
3        4  Man Who Knew Too Little, The (1997)            1
4        5                        Target (1995)            1


---

## **Submission Requirements:**

To successfully complete this assignment, ensure that you submit the following:


### **1. Jupyter Notebook Submission**
- Submit a **fully completed Jupyter Notebook** that includes:
  - **All implemented recommendation functions** (user-based, item-based, and random walk-based recommendations).
  - **Code explanations** in markdown cells to describe each step.
  - **Results and insights** from running your recommendation models.


### **2. Explanation of Pixie-Inspired Algorithms (3-5 Paragraphs)**
- Write a **detailed explanation** of **Pixie-inspired random walk algorithms** used for recommendations.
- Your explanation should cover:
  - What **Pixie-inspired recommendation systems** are.
  - How **random walks** help in identifying relevant recommendations.
  - Any real-world applications of such algorithms in industry.


### **3. Report for the Submitted Notebook**
Your report should be structured as follows:

#### **Title: Movie Recommendation System Report**

#### **1. Introduction**
- Briefly introduce **movie recommendation systems** and why they are important.
- Explain the **different approaches used** (user-based, item-based, random-walk).

#### **2. Dataset Description**
- Describe the **MovieLens 100K dataset**:
  - Number of users, movies, and ratings.
  - What features were used.
  - Any preprocessing performed.

#### **3. Methodology**
- Explain the three recommendation techniques implemented:
  - **User-based collaborative filtering** (how user similarity was calculated).
  - **Item-based collaborative filtering** (how item similarity was determined).
  - **Random-walk-based Pixie algorithm** (why graph-based approaches are effective).
  
#### **4. Implementation Details**
- Discuss the steps taken to build the functions.
- Describe how the **adjacency list graph** was created.
- Explain how **random walks** were performed and how visited movies were ranked.

#### **5. Results and Evaluation**
- Present **example outputs** from each recommendation approach.
- Compare the different methods in terms of accuracy and usefulness.
- Discuss any **limitations** in the implementation.

#### **6. Conclusion**
- Summarize the key takeaways from the project.
- Discuss potential improvements (e.g., **hybrid models, additional features**).
- Suggest real-world applications of the methods used.

### **Submission Instructions**

- Submit `.zip` file consisting of Jupyter Notebook and all the datafiles (provided) and the ones saved [i.e. `users.csv`, `movies.csv` and `ratings.csv`]. Also, include the Report and Pixie Algorithm explanation document.
- [`Bonus 10 Points`] **Upload your Jupyter Notebook, Explanation Document, and Report** to your GitHub repository.
- Ensure the repository is public and contains:
  - `users.csv`, `movies.csv` and `ratings.csv` [These are the Dataframes which were created in part 1. Save and export them as a `.csv` file]
  - `Movie_Recommendation.ipynb`
  - `Pixie_Algorithm_Explanation.pdf` or `.md`
  - `Recommendation_Report.pdf` or `.md`
- **Submit the GitHub repository link in the cell below.**


#### **Example Submission Format**
```text
GitHub Repository: https://github.com/username/Movie-Recommendation
```

In [ ]:
# Submit the Github Link here:


### **Grading Rubric: ITCS 6162 - Data Mining Assignment**


| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **Part 1: Exploring and Cleaning Data (15 pts)**  | Properly loads `u.user`, `u.movies`, and `u.item` datasets into DataFrames | 5 |
|                                           | Handles missing values, duplicates, and inconsistencies appropriately | 5 |
|                                           | Saves the cleaned datasets into CSV files: `users.csv`, `movies.csv`, `ratings.csv` | 5 |
| **Part 2: Collaborative Filtering-Based Recommendation (30 pts)** | Implements user-based collaborative filtering correctly | 10 |
|                                           | Implements item-based collaborative filtering correctly | 10 |
|                                           | Computes similarity measures accurately and provides valid recommendations | 10 |
| **Part 3: Graph-Based Recommender (Pixie-Inspired Algorithm) (35 pts)** | Constructs adjacency lists properly from user-movie interactions | 10 |
|                                           | Implements weighted random walk-based recommendation correctly | 15 |
|                                           | Explains and justifies the algorithm design choices (Pixie-inspired) | 10 |
| **Code Quality & Documentation (10 pts)** | Code is well-structured, efficient, and follows best practices | 5 |
|                                           | Markdown explanations and comments are clear and enhance understanding | 5 |
| **Results & Interpretation (5 pts)**      | Provides meaningful insights from the recommendation system's output | 5 |
| **Submission & Report (5 pts)**          | Submits all required files in the correct format (ZIP file with Jupyter notebook, processed CSV files, and project report) | 5 |
| **Total**                                 |                              | 100 |

#### **Bonus (10 pts)**
| **Category**                              | **Criteria**                                                     | **Points** |
|-------------------------------------------|----------------------------------------------------------------|------------|
| **GitHub Submission**                     | Provides a well-documented GitHub repository with CSV files, a structured README, and a properly formatted Jupyter Notebook | 10 |